In [15]:
import math
from collections import defaultdict, deque
import heapq

# Hardcoded city positions (x, y coordinates)
positions = {
    "Copenhagen": (687.7241, 1323.35),
    "Hamburg": (774.7866, 1175.2741),
    "Berlin": (626.85, 1131.8125),
    "Warsaw": (339.4741, 1149.225),
    "Amsterdam": (957.6875, 1096.9875),
    "Brussel": (983.7366, 992.5125),
    "Prague": (574.6125, 975.1),
    "Paris": (1062.1625, 870.625),
    "Bern": (853.2125, 818.3875),
    "Munich": (679.0875, 835.8),
    "Vienna": (478.7741, 870.625),
    "Budapest": (365.6625, 844.4366),
    "Belgrade": (269.8241, 687.7241),
    "Trieste": (548.4241, 722.5491),
    "Genoa": (783.5625, 609.4375),
    "Rome": (600.6616, 452.725),
    "Madrid": (1375.5875, 383.075),
    "Naples": (504.9625, 400.4875),
    "Lisbon": (1619.3625, 296.0125),
}

# Hardcoded road distances (city1, city2, distance)
roads_data = [
    ("Copenhagen", "Hamburg", 180),
    ("Hamburg", "Amsterdam", 338),
    ("Hamburg", "Berlin", 182),
    ("Berlin", "Bern", 628),
    ("Berlin", "Prague", 219),
    ("Berlin", "Warsaw", 345),
    ("Warsaw", "Prague", 479),
    ("Warsaw", "Vienna", 464),
    ("Warsaw", "Budapest", 394),
    ("Amsterdam", "Munich", 526),
    ("Amsterdam", "Bern", 558),
    ("Amsterdam", "Brussel", 164),
    ("Brussel", "Bern", 497),
    ("Brussel", "Genoa", 740),
    ("Brussel", "Paris", 225),
    ("Prague", "Vienna", 185),
    ("Prague", "Munich", 174),
    ("Paris", "Genoa", 629),
    ("Paris", "Madrid", 805),
    ("Bern", "Munich", 311),
    ("Bern", "Trieste", 489),
    ("Bern", "Genoa", 304),
    ("Bern", "Madrid", 1104),
    ("Munich", "Vienna", 280),
    ("Munich", "Rome", 582),
    ("Vienna", "Budapest", 155),
    ("Vienna", "Trieste", 317),
    ("Vienna", "Belgrade", 501),
    ("Budapest", "Trieste", 384),
    ("Budapest", "Belgrade", 263),
    ("Belgrade", "Trieste", 403),
    ("Trieste", "Genoa", 361),
    ("Trieste", "Rome", 442),
    ("Genoa", "Madrid", 951),
    ("Genoa", "Rome", 328),
    ("Rome", "Naples", 134),
    ("Madrid", "Lisbon", 339),
]

# Convert roads_data into a graph structure
roads = defaultdict(list)
for city1, city2, dist in roads_data:
    roads[city1].append((city2, dist))
    roads[city2].append((city1, dist))  # Roads are bidirectional

# Heuristic function for Greedy and A* Search (Euclidean distance)
def heuristic(city, goal, positions, factor=1.0):
    """
    Calculate the Euclidean distance between two cities using their coordinates.
    This is used as the heuristic for Greedy and A* Search.
    """
    x1, y1 = positions[city]
    x2, y2 = positions[goal]
    return factor * math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Depth First Search (DFS)
def dfs(start, goal, roads):
    """
    Perform Depth First Search to find a path from start to goal.
    Returns the number of nodes expanded, the total cost, and the path.
    """
    stack = [(start, [start], 0)]  # Stack stores (current_city, path, cost)
    visited = set()
    nodes_expanded = 0

    while stack:
        nodes_expanded += 1
        current, path, cost = stack.pop()
        if current == goal:
            return nodes_expanded, cost, path
        if current not in visited:
            visited.add(current)
            for neighbor, distance in roads[current]:
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor], cost + distance))
    return None  # No path found

# Breadth First Search (BFS)
def bfs(start, goal, roads):
    """
    Perform Breadth First Search to find a path from start to goal.
    Returns the number of nodes expanded, the total cost, and the path.
    """
    queue = deque([(start, [start], 0)])  # Queue stores (current_city, path, cost)
    visited = set()
    nodes_expanded = 0

    while queue:
        nodes_expanded += 1
        current, path, cost = queue.popleft()
        if current == goal:
            return nodes_expanded, cost, path
        if current not in visited:
            visited.add(current)
            for neighbor, distance in roads[current]:
                if neighbor not in visited:
                    queue.append((neighbor, path + [neighbor], cost + distance))
    return None  # No path found

# Iterative Deepening Search (IDS)
def ids(start, goal, roads, max_depth=100):
    """
    Perform Iterative Deepening Search to find a path from start to goal.
    Returns the number of nodes expanded, the total cost, and the path.
    """
    def dls(current, goal, depth, path, cost, visited):
        """
        Depth-Limited Search (DLS) helper function for IDS.
        """
        if depth == 0:
            return None
        if current == goal:
            return cost, path
        visited.add(current)
        for neighbor, distance in roads[current]:
            if neighbor not in visited:
                result = dls(neighbor, goal, depth - 1, path + [neighbor], cost + distance, visited)
                if result is not None:
                    return result
        return None

    nodes_expanded = 0
    for depth in range(max_depth):
        visited = set()
        result = dls(start, goal, depth, [start], 0, visited)
        nodes_expanded += len(visited)
        if result is not None:
            return nodes_expanded, result[0], result[1]
    return None  # No path found

# Uniform Cost Search (UCS)
def ucs(start, goal, roads):
    """
    Perform Uniform Cost Search to find a path from start to goal.
    Returns the number of nodes expanded, the total cost, and the path.
    """
    heap = [(0, start, [start])]  # Heap stores (cost, current_city, path)
    visited = set()
    nodes_expanded = 0

    while heap:
        nodes_expanded += 1
        cost, current, path = heapq.heappop(heap)
        if current == goal:
            return nodes_expanded, cost, path
        if current not in visited:
            visited.add(current)
            for neighbor, distance in roads[current]:
                if neighbor not in visited:
                    heapq.heappush(heap, (cost + distance, neighbor, path + [neighbor]))
    return None  # No path found

# Greedy Search
def greedy(start, goal, roads, positions, factor=1.0):
    """
    Perform Greedy Best-First Search with a heuristic scaling factor.
    """
    heap = [(heuristic(start, goal, positions, factor), start, [start], 0)]  
    visited = set()
    nodes_expanded = 0

    while heap:
        nodes_expanded += 1
        _, current, path, cost = heapq.heappop(heap)
        if current == goal:
            return nodes_expanded, cost, path
        if current not in visited:
            visited.add(current)
            for neighbor, distance in roads[current]:
                if neighbor not in visited:
                    heapq.heappush(heap, (heuristic(neighbor, goal, positions, factor), neighbor, path + [neighbor], cost + distance))
    return None  # No path found

# A* Search
def astar(start, goal, roads, positions, factor=1.0):
    """
    Perform A* Search with a heuristic scaling factor.
    """
    def f_cost(cost, city):
        """
        Calculate the f-cost for A* Search: f(n) = g(n) + h(n), with a scaling factor.
        """
        return cost + heuristic(city, goal, positions, factor)

    heap = [(f_cost(0, start), 0, start, [start])]  
    visited = set()
    nodes_expanded = 0

    while heap:
        nodes_expanded += 1
        _, cost, current, path = heapq.heappop(heap)
        if current == goal:
            return nodes_expanded, cost, path
        if current not in visited:
            visited.add(current)
            for neighbor, distance in roads[current]:
                if neighbor not in visited:
                    heapq.heappush(heap, (f_cost(cost + distance, neighbor), cost + distance, neighbor, path + [neighbor]))
    return None  # No path found

# User Interface
def main():
    """
    Main function to interact with the user and run the search algorithms.
    """
    start = input("Enter the starting city: ")
    goal = input("Enter the goal city: ")
    print("Choose a search strategy:")
    print("1. Depth First Search")
    print("2. Breadth First Search")
    print("3. Iterative Deepening")
    print("4. Uniform Cost Search")
    print("5. Greedy Search")
    print("6. A* Search")
    choice = int(input("Enter your choice: "))

    factor = 1.0  # Default heuristic factor
    if choice in {5, 6}:  # If using Greedy Search or A*
        factor = float(input("Enter heuristic scaling factor (0.5, 1.0, or 1.5): "))

    if choice == 1:
        result = dfs(start, goal, roads)
    elif choice == 2:
        result = bfs(start, goal, roads)
    elif choice == 3:
        result = ids(start, goal, roads)
    elif choice == 4:
        result = ucs(start, goal, roads)
    elif choice == 5:
        result = greedy(start, goal, roads, positions, factor)
    elif choice == 6:
        result = astar(start, goal, roads, positions, factor)
    else:
        print("Invalid choice")
        return

    if result:
        nodes_expanded, cost, path = result
        print(f"Nodes expanded: {nodes_expanded}")
        print(f"Total cost: {cost}")
        print(f"Path: {' -> '.join(path)}")
    else:
        print("No path found")

# Run the program
if __name__ == "__main__":
    main()

Enter the starting city:  Paris
Enter the goal city:  Vienna


Choose a search strategy:
1. Depth First Search
2. Breadth First Search
3. Iterative Deepening
4. Uniform Cost Search
5. Greedy Search
6. A* Search


Enter your choice:  6
Enter heuristic scaling factor (0.5, 1.0, or 1.5):  1.5


Nodes expanded: 5
Total cost: 1195
Path: Paris -> Brussel -> Amsterdam -> Munich -> Vienna
